<a href="https://colab.research.google.com/github/EAName/yelp-reviews/blob/main/Yelp_Top_Words_for_Each_Business_RA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import os
import random
import numpy as np
import pickle 
import re,string

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize 

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD, LatentDirichletAllocation
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, silhouette_score
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, KFold
from sklearn.manifold import MDS
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import LinearSVC

import xgboost as xgb

from sklearn.model_selection import train_test_split 
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold 
from sklearn.model_selection import cross_val_predict


import seaborn as sns
import matplotlib.pyplot as plt

from gensim.models import Word2Vec,LdaMulticore, TfidfModel, CoherenceModel
from gensim import corpora
from gensim import similarities
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import LsiModel,LdaModel

from keras.preprocessing.text import Tokenizer
#from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, SpatialDropout1D, GRU
from keras.callbacks import EarlyStopping
from keras.initializers import Constant
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam, Nadam, RMSprop
from keras import Input

from scipy import sparse

from IPython.display import display

In [2]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Load Reviews Dataset

In [5]:
df =pd.read_csv('/content/gdrive/MyDrive/Data/yelp_reviews_clean_version4_st_louis.csv')

In [6]:
df['stars'].value_counts()

5.0    124247
4.0     73648
3.0     35345
1.0     32873
2.0     26263
Name: stars, dtype: int64

In [7]:
df = df[df['stars'].isin([1.0, 4.0, 5.0])] 

In [8]:
df['stars'].value_counts()

5.0    124247
4.0     73648
1.0     32873
Name: stars, dtype: int64

In [9]:
def clean_doc(doc): 
    #split review into individual words
    tokens=doc.split()
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    # remove punctuation from each word
    tokens = [re_punc.sub('', w) for w in tokens]

    #word lemmzatization    
    lemm = WordNetLemmatizer()
    tokens=[lemm.lemmatize(word) for word in tokens]

    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 3]
    #lowercase all words
    tokens = [word.lower() for word in tokens]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]     

    # filter out common words
    tokens = [w for w in tokens if not w in ['louis', 'restaurant', 'also', 'youre']] 
    # # word stemming    
    # ps=PorterStemmer()
    # tokens=[ps.stem(word) for word in tokens]

    return tokens

def final_processed_text(doc):
    #this is a function to join the processed text back
    ' '.join(doc)
    return doc

In [10]:
df['cleaned_text'] = df['text'].apply(lambda x: clean_doc(x))

final_processed_text = [' '.join(x) for x in df['cleaned_text'].tolist()]
df['final_processed_text'] = final_processed_text     #add to dataframe

## Top Negative Words

In [22]:
df_negative = df[df['stars'].isin([1.0])] 

In [23]:
len(df_negative)

32873

In [24]:
titles = list(range(len(df_negative)))
cv = CountVectorizer(binary=False, ngram_range = (1,1), max_features = 2000)
count_matrix = cv.fit_transform(df_negative['final_processed_text'])

words = cv.get_feature_names()
matrix=pd.DataFrame(count_matrix.toarray(), columns=words, index=titles)

In [25]:
df_negative['business_id'].nunique()

2859

In [26]:
# add business_id to count_matrix
business_negative = df_negative['business_id'].tolist()
matrix['business_id'] = business_negative

In [27]:
# sum of words for each business
df_negative_sum = matrix.groupby('business_id').sum()

In [28]:
len(df_negative_sum)

2859

In [29]:
# create empty dataframe to add top negative words
df_negative_top_words = df_negative_sum
df_negative_top_words['business_id'] = df_negative_sum.index.tolist()
df_negative_top_words = df_negative_top_words.iloc[:, -1:]

In [30]:
# sum of words for each business
df_negative_sum = matrix.groupby('business_id').sum()

In [31]:
for index, row in df_negative_sum.iterrows():
  sorted_words = row.sort_values(ascending=False)
  # print(sorted_words)
  # print(sorted_words[0])
  # print(sorted_words.index[0])
  # print(index)
  for i in range(1,16):
    df_negative_top_words.loc[index,"negative_word{0}".format(i)] = sorted_words.index[i]
    df_negative_top_words.loc[index, "negative_word_count{0}".format(i)] = sorted_words[i]

In [32]:
df_negative_top_words.head(10)

,business_id,negative_word1,negative_word_count1,negative_word2,negative_word_count2,negative_word3,negative_word_count3,negative_word4,negative_word_count4,negative_word5,...,negative_word11,negative_word_count11,negative_word12,negative_word_count12,negative_word13,negative_word_count13,negative_word14,negative_word_count14,negative_word15,negative_word_count15
business_id,,,,,,,,,,,,,,,,,,,,,
-1wnOZFsvsJPEXh_hWcmMA,-1wnOZFsvsJPEXh_hWcmMA,ordered,2.0,server,2.0,finally,2.0,food,2.0,minute,...,asked,1.0,leaving,1.0,ticket,1.0,recommended,1.0,high,1.0
-2gmbMDzKgYZ_8DOnJtPyw,-2gmbMDzKgYZ_8DOnJtPyw,order,12.0,ordered,9.0,time,8.0,customer,7.0,would,...,owner,5.0,said,5.0,service,5.0,minute,5.0,rice,5.0
-6OjnX3ZdDOhHxWR60wysg,-6OjnX3ZdDOhHxWR60wysg,table,13.0,drink,12.0,time,11.0,server,10.0,would,...,ordered,8.0,steak,7.0,ever,7.0,place,7.0,asked,7.0
-6jVwUuXC1-dyr2YRCEU6Q,-6jVwUuXC1-dyr2YRCEU6Q,order,28.0,chicken,23.0,never,16.0,said,14.0,spicy,...,good,11.0,curry,10.0,give,10.0,asked,10.0,time,10.0
-6kNhGurAbdNQanqbAaoMg,-6kNhGurAbdNQanqbAaoMg,wing,5.0,sauce,4.0,hotel,3.0,extra,3.0,disappointed,...,told,2.0,front,2.0,never,2.0,crust,2.0,flavor,2.0
-78Va3llKcPbL6SW4is8lw,-78Va3llKcPbL6SW4is8lw,sauce,15.0,time,14.0,pasta,13.0,came,10.0,said,...,dont,8.0,would,7.0,take,7.0,place,7.0,customer,7.0
-8jIGHKz1-Tnv2X85kxbvQ,-8jIGHKz1-Tnv2X85kxbvQ,order,5.0,time,4.0,dont,4.0,ordered,4.0,changed,...,first,2.0,life,2.0,hour,2.0,noodle,2.0,would,2.0
-8xqG21DUzav7yIp5lS6jw,-8xqG21DUzav7yIp5lS6jw,ever,3.0,cheese,2.0,enough,2.0,extremely,2.0,good,...,left,1.0,slow,1.0,thin,1.0,service,1.0,last,1.0
-9Kxik9BRNkft5rPcVvSsw,-9Kxik9BRNkft5rPcVvSsw,food,36.0,time,34.0,service,34.0,wing,33.0,back,...,would,20.0,drink,18.0,even,17.0,never,17.0,ordered,17.0


In [33]:
df_negative_top_words.to_csv('/content/gdrive/MyDrive/Data/negative_top_words.csv', index = False)

## Top Positive Words

In [11]:
df = df[df['stars'].isin([4.0, 5.0])] 

In [12]:
len(df)

197895

In [13]:
titles = list(range(len(df)))
cv = CountVectorizer(binary=False, ngram_range = (1,1), max_features = 2000)
count_matrix = cv.fit_transform(df['final_processed_text'])

words = cv.get_feature_names()
matrix=pd.DataFrame(count_matrix.toarray(), columns=words, index=titles)

In [14]:
df['business_id'].nunique()

3163

In [15]:
# add business_id to count_matrix
business_positive = df['business_id'].tolist()
matrix['business_id'] = business_positive

In [16]:
# sum of words for each business
df_positive_sum = matrix.groupby('business_id').sum()

In [17]:
len(df_positive_sum)

3163

In [18]:
# create empty dataframe to add top positive words
df_positive_top_words = df_positive_sum
df_positive_top_words['business_id'] = df_positive_sum.index.tolist()
df_positive_top_words = df_positive_top_words.iloc[:, -1:]

In [19]:
# sum of words for each business
df_positive_sum = matrix.groupby('business_id').sum()

In [20]:
for index, row in df_positive_sum.iterrows():
  sorted_words = row.sort_values(ascending=False)
  # print(sorted_words)
  # print(sorted_words[0])
  # print(sorted_words.index[0])
  # print(index)
  for i in range(1,16):
    df_positive_top_words.loc[index,"positive_word{0}".format(i)] = sorted_words.index[i]
    df_positive_top_words.loc[index, "positive_word_count{0}".format(i)] = sorted_words[i]

In [21]:
df_positive_top_words.head(10)

,business_id,positive_word1,positive_word_count1,positive_word2,positive_word_count2,positive_word3,positive_word_count3,positive_word4,positive_word_count4,positive_word5,...,positive_word11,positive_word_count11,positive_word12,positive_word_count12,positive_word13,positive_word_count13,positive_word14,positive_word_count14,positive_word15,positive_word_count15
business_id,,,,,,,,,,,,,,,,,,,,,
-1wnOZFsvsJPEXh_hWcmMA,-1wnOZFsvsJPEXh_hWcmMA,food,5.0,taco,4.0,burger,4.0,fresh,4.0,delicious,...,staff,3.0,friendly,3.0,special,3.0,beer,3.0,menu,2.0
-2gmbMDzKgYZ_8DOnJtPyw,-2gmbMDzKgYZ_8DOnJtPyw,chicken,8.0,crab,8.0,place,7.0,rice,7.0,chinese,...,meal,4.0,order,4.0,really,4.0,fresh,3.0,ordered,3.0
-6OjnX3ZdDOhHxWR60wysg,-6OjnX3ZdDOhHxWR60wysg,good,38.0,food,32.0,place,31.0,time,28.0,really,...,chicken,17.0,menu,16.0,would,16.0,ordered,16.0,service,16.0
-6jVwUuXC1-dyr2YRCEU6Q,-6jVwUuXC1-dyr2YRCEU6Q,chicken,138.0,indian,130.0,good,111.0,place,106.0,lunch,...,rice,50.0,great,48.0,time,47.0,masala,43.0,order,42.0
-6kNhGurAbdNQanqbAaoMg,-6kNhGurAbdNQanqbAaoMg,imos,14.0,cheese,7.0,provel,7.0,location,5.0,like,...,people,4.0,love,4.0,sauce,4.0,change,3.0,best,3.0
-78Va3llKcPbL6SW4is8lw,-78Va3llKcPbL6SW4is8lw,food,70.0,salad,52.0,good,51.0,place,46.0,service,...,delicious,29.0,well,28.0,sauce,28.0,best,27.0,ordered,26.0
-8jIGHKz1-Tnv2X85kxbvQ,-8jIGHKz1-Tnv2X85kxbvQ,good,7.0,always,7.0,chinese,7.0,chicken,7.0,love,...,order,4.0,service,4.0,menu,4.0,broccoli,4.0,soup,3.0
-8xqG21DUzav7yIp5lS6jw,-8xqG21DUzav7yIp5lS6jw,place,24.0,great,22.0,time,17.0,love,15.0,good,...,best,10.0,like,10.0,first,9.0,crust,9.0,cheese,9.0
-9Kxik9BRNkft5rPcVvSsw,-9Kxik9BRNkft5rPcVvSsw,wing,17.0,good,16.0,service,15.0,great,13.0,chicken,...,back,7.0,server,7.0,friendly,7.0,favorite,6.0,like,6.0


In [22]:
df_positive_top_words.to_csv('/content/gdrive/MyDrive/Data/positive_top_words.csv', index = False)

## Combined Dataframe of Top Positive and Negative Words

In [ ]:
df_positive_top_words =pd.read_csv('/content/gdrive/MyDrive/Data/positive_top_words.csv')

In [25]:
df_positive_top_words = df_positive_top_words.reset_index(drop=True)

In [23]:
df_negative_top_words =pd.read_csv('/content/gdrive/MyDrive/Data/negative_top_words.csv')

In [26]:
top_words = pd.merge(df_positive_top_words, df_negative_top_words, on ='business_id')

In [27]:
top_words.head(10)

,business_id,positive_word1,positive_word_count1,positive_word2,positive_word_count2,positive_word3,positive_word_count3,positive_word4,positive_word_count4,positive_word5,...,negative_word11,negative_word_count11,negative_word12,negative_word_count12,negative_word13,negative_word_count13,negative_word14,negative_word_count14,negative_word15,negative_word_count15
0,-1wnOZFsvsJPEXh_hWcmMA,food,5.0,taco,4.0,burger,4.0,fresh,4.0,delicious,...,asked,1.0,leaving,1.0,ticket,1.0,recommended,1.0,high,1.0
1,-2gmbMDzKgYZ_8DOnJtPyw,chicken,8.0,crab,8.0,place,7.0,rice,7.0,chinese,...,owner,5.0,said,5.0,service,5.0,minute,5.0,rice,5.0
2,-6OjnX3ZdDOhHxWR60wysg,good,38.0,food,32.0,place,31.0,time,28.0,really,...,ordered,8.0,steak,7.0,ever,7.0,place,7.0,asked,7.0
3,-6jVwUuXC1-dyr2YRCEU6Q,chicken,138.0,indian,130.0,good,111.0,place,106.0,lunch,...,good,11.0,curry,10.0,give,10.0,asked,10.0,time,10.0
4,-6kNhGurAbdNQanqbAaoMg,imos,14.0,cheese,7.0,provel,7.0,location,5.0,like,...,told,2.0,front,2.0,never,2.0,crust,2.0,flavor,2.0
5,-78Va3llKcPbL6SW4is8lw,food,70.0,salad,52.0,good,51.0,place,46.0,service,...,dont,8.0,would,7.0,take,7.0,place,7.0,customer,7.0
6,-8jIGHKz1-Tnv2X85kxbvQ,good,7.0,always,7.0,chinese,7.0,chicken,7.0,love,...,first,2.0,life,2.0,hour,2.0,noodle,2.0,would,2.0
7,-8xqG21DUzav7yIp5lS6jw,place,24.0,great,22.0,time,17.0,love,15.0,good,...,left,1.0,slow,1.0,thin,1.0,service,1.0,last,1.0
8,-9Kxik9BRNkft5rPcVvSsw,wing,17.0,good,16.0,service,15.0,great,13.0,chicken,...,would,20.0,drink,18.0,even,17.0,never,17.0,ordered,17.0
9,-Bsw0AjWemVa8D_7g7-9Kg,drink,16.0,music,16.0,great,12.0,sushi,9.0,service,...,know,10.0,manager,9.0,dont,9.0,bathroom,9.0,ordered,9.0


In [32]:
top_words=pd.merge(df_positive_top_words,df_negative_top_words, left_on='business_id', right_on='business_id', how='outer')

In [34]:
top_words.head(10)

,business_id,positive_word1,positive_word_count1,positive_word2,positive_word_count2,positive_word3,positive_word_count3,positive_word4,positive_word_count4,positive_word5,...,negative_word11,negative_word_count11,negative_word12,negative_word_count12,negative_word13,negative_word_count13,negative_word14,negative_word_count14,negative_word15,negative_word_count15
0,-1wnOZFsvsJPEXh_hWcmMA,food,5.0,taco,4.0,burger,4.0,fresh,4.0,delicious,...,asked,1.0,leaving,1.0,ticket,1.0,recommended,1.0,high,1.0
1,-2gmbMDzKgYZ_8DOnJtPyw,chicken,8.0,crab,8.0,place,7.0,rice,7.0,chinese,...,owner,5.0,said,5.0,service,5.0,minute,5.0,rice,5.0
2,-6OjnX3ZdDOhHxWR60wysg,good,38.0,food,32.0,place,31.0,time,28.0,really,...,ordered,8.0,steak,7.0,ever,7.0,place,7.0,asked,7.0
3,-6jVwUuXC1-dyr2YRCEU6Q,chicken,138.0,indian,130.0,good,111.0,place,106.0,lunch,...,good,11.0,curry,10.0,give,10.0,asked,10.0,time,10.0
4,-6kNhGurAbdNQanqbAaoMg,imos,14.0,cheese,7.0,provel,7.0,location,5.0,like,...,told,2.0,front,2.0,never,2.0,crust,2.0,flavor,2.0
5,-78Va3llKcPbL6SW4is8lw,food,70.0,salad,52.0,good,51.0,place,46.0,service,...,dont,8.0,would,7.0,take,7.0,place,7.0,customer,7.0
6,-8jIGHKz1-Tnv2X85kxbvQ,good,7.0,always,7.0,chinese,7.0,chicken,7.0,love,...,first,2.0,life,2.0,hour,2.0,noodle,2.0,would,2.0
7,-8xqG21DUzav7yIp5lS6jw,place,24.0,great,22.0,time,17.0,love,15.0,good,...,left,1.0,slow,1.0,thin,1.0,service,1.0,last,1.0
8,-9Kxik9BRNkft5rPcVvSsw,wing,17.0,good,16.0,service,15.0,great,13.0,chicken,...,would,20.0,drink,18.0,even,17.0,never,17.0,ordered,17.0
9,-Bsw0AjWemVa8D_7g7-9Kg,drink,16.0,music,16.0,great,12.0,sushi,9.0,service,...,know,10.0,manager,9.0,dont,9.0,bathroom,9.0,ordered,9.0


In [33]:
top_words.shape

(3181, 61)

## Load Business Dataset

In [35]:
df_stlouis =pd.read_csv('/content/gdrive/MyDrive/Data/business_df_stlouis.csv')

In [36]:
df_stlouis.shape

(3181, 69)

## Combine Business Dataset and Top Words

In [37]:
top_words_by_business = pd.merge(top_words, df_stlouis, on ='business_id')

In [48]:
top_words_by_business.shape

(3181, 129)

In [49]:
top_words_by_business.to_csv('/content/gdrive/MyDrive/Data/top_words_by_business.csv', index = False)